# GPU acceleration

## Overview

* **Why to bother with GPU computing in 2024**
  * HPC and Supercomputing is GPU-accelerated
  * When Julia overcomes the two-language barrier

* **GPU computing Fast-Forward**
  * Array vs Kernel programming
  * Performance considerations

* **Going multi-GPUs**
  * MPI + GPUs

### Why to still bother with GPU computing in 2024
- It's around for more than a decade
- It shows massive performance gain compared to serial CPU computing
- First exascale supercomputer, Frontier, is full of GPUs

<img src="imgs/frontier.png" width=400px>

### Performance that matters

<img src="imgs/cpu_gpu_evo.png" width=600px>

Taking a look at a recent GPU and CPU:
- Nvidia Tesla A100 GPU
- AMD EPYC "Rome" 7282 (16 cores) CPU

| Device         | TFLOP/s (FP64) | Memory BW TB/s | Imbalance (FP64)     |
| :------------: | :------------: | :------------: | :------------------: |
| Tesla A100     | 9.7            | 1.55           | 9.7 / 1.55  × 8 = 50 |
| AMD EPYC 7282  | 0.7            | 0.085          | 0.7 / 0.085 × 8 = 66 |

**Meaning:** we can do about 50 floating point operations per number accessed from main memory.
Floating point operations are "for free" when we work in memory-bounded regimes.

👉 Requires re-thinking the numerical implementation and solution strategies

Unfortunately, the cost of evaluating a first derivative $∂A / ∂x$ in, e.g., diffusive flux calculations using finite-differences:

`q[ix] = -D * (A[ix+1] - A[ix]) / dx`

consists of:
- 1 read (`A`) + 1 write (`q`) => $2 × 8$ = **16 Bytes transferred**
- 1 addition + 1 multiplication + 1 division => **3 floating point operations**

👉 assuming `D`, `dx` are scalars, `q` and `A` are arrays of `Float64` (read from main memory)

### Performance that matters - an example
Not yet convinced? Let's have a look at an example.

Let's assess how close from memory copy (1400 GB/s) we can get solving a 2D diffusion problem on an Nvidia Tesla A100 GPU.

$$ \frac{\partial C}{\partial t} = \frac{\partial^2 C}{\partial x^2} + \frac{\partial^2 C}{\partial y^2} $$

👉 Let's test the performance using a simple script.

### Measuring GPU performance

Load modules:

In [3]:
using CUDA
using BenchmarkTools
using Printf

Memory copy function to measure the "peak" memory throughput:

In [4]:
function mycopy!(A, B)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix <= size(A, 1) && iy <= size(A, 2)
        @inbounds A[ix, iy] = B[ix, iy] + 1
    end
    return
end

mycopy! (generic function with 1 method)

Laplacian kernel using the finite difference method (FDM):

In [5]:
function laplacian!(A, B, dt, _dx2, _dy2)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix <= size(A, 1) - 2 && iy <= size(A, 2) - 2
        @inbounds A[ix+1, iy+1] = B[ix+1, iy+1] + dt *
            ((B[ix+2, iy+1] - 2 * B[ix+1, iy+1] + B[ix, iy+1]) * _dx2 +
             (B[ix+1, iy+2] - 2 * B[ix+1, iy+1] + B[ix+1, iy]) * _dy2)
    end
    return
end

laplacian! (generic function with 1 method)

Let's test the performance!

In [8]:
# if the array size is too small, the GPU will not be fully utilized
nx = ny = 512 * 32
A = CUDA.rand(Float64, nx, ny)
B = CUDA.rand(Float64, nx, ny)

_dx2 = _dy2 = dt = rand()

# launch configuration
nthreads = (16, 16)
nblocks  = cld.((nx, ny), nthreads)

# measure the execution times
time_copy = @belapsed CUDA.@sync @cuda threads=nthreads blocks=nblocks mycopy!(A, B)
time_lapl = @belapsed CUDA.@sync @cuda threads=nthreads blocks=nblocks laplacian!(A, B, dt, _dx2, _dy2)

# effective memory throughput (1 read + 1 write per element)
Teff_copy = 2 * nx * ny * sizeof(Float64) / time_copy / 1e9
Teff_lapl = 2 * nx * ny * sizeof(Float64) / time_lapl / 1e9

# compute theoretical peak memory bandwidth
dev = CUDA.device()

bus_width       = CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_GLOBAL_MEMORY_BUS_WIDTH) |> Float64 # in bits
clock_rate      = CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MEMORY_CLOCK_RATE)       |> Float64 # in kHz
rate_multiplier = 2 # 2 for HBM2/DDR, 4 for HBM3/GDDR5, 8 for GDDR6

Teff_peak = bus_width * clock_rate * rate_multiplier / 1e6 / 8

# report results
@printf("Effective memory throughput (copy)      : %.2f GB/s\n", Teff_copy)
@printf("Effective memory throughput (laplacian) : %.2f GB/s\n", Teff_lapl)
@printf("Theoretical peak memory throughput      : %.2f GB/s\n", Teff_peak)

@printf("\nWow 🚀! Laplacian runs at:\n")
@printf("   %.2f%% of copy speed\n"           , 100 * Teff_lapl / Teff_copy)
@printf("   %.2f%% of peak memory bandwidth\n", 100 * Teff_lapl / Teff_peak)
@printf("on a %s device\n", CUDA.name(dev))

Effective memory throughput (copy)      : 1335.85 GB/s
Effective memory throughput (laplacian) : 1303.32 GB/s
Theoretical peak memory throughput      : 1555.20 GB/s

Wow 🚀! Laplacian runs at:
   97.56% of copy speed
   83.80% of peak memory bandwidth
on a NVIDIA A100-SXM4-40GB device


### GPU kernel programming

We'll get started with a brief overview of the Nvidia GPU architecture and how to program it.

The Nvidia general purpose GPUs can be programmed using the CUDA language extension. CUDA is accessible in Julia via [CUDA.jl](https://cuda.juliagpu.org/stable/), which exposes most of the native CUDA features to the Julia ecosystem.

In the CUDA programming model, `blocks` of `threads` compose the `grid`. In our implementation, we want to map one thread to each finite-difference cell of the 2D Cartesian domain.

The figure hereafter depicts the relation between the CUDA domain and the finite-difference domain:

<img src="imgs/cuda_grid.png" width=600px>

**Playing with GPUs: the rules**

- Current GPUs allow typically a maximum of 1024 threads per block.

- The maximum number of blocks allowed is huge; computing the largest possible array on the GPU will make you run out of device memory (currently 16-80 GB) before hitting the maximal number of blocks when selecting sensible kernel launch parameters (usually threads per block >= 128).

- Threads, blocks and grid have 3D "Cartesian" topology, which is very useful for 1D, 2D and 3D Cartesian finite-difference domains.

### Multi-GPU

#### GPU - MPI ranks mapping
The challenging part is to run on multiple GPUs using MPI. To achieve this, we need to map node-local MPI ranks to GPU IDs.

This can be achieved in Julia using MPI.jl and CUDA.jl by
```julia
comm   = MPI.COMM_WORLD
rank   = MPI.Comm_rank(comm)
comm_l = MPI.Comm_split_type(comm, MPI.COMM_TYPE_SHARED, rank)
rank_l = MPI.Comm_rank(comm_l)
gpu_id = CUDA.device!(rank_l)
```

#### GPU-aware MPI

On modern supercomputers, one has access to GPU-aware MPI. GPU aware-MPI allows to directly exchange GPU memory by-passing an explicit host copy.

The file [`multigpu.jl`](./multigpu.jl) implements this and would check that GPU-aware MPI works:

In [2]:
run_cmd = `mpiexecjl -n 4 -G 4 --nodes 1 --qos regular --constraint gpu --gpus 4 --account=ntrain1 --project julia multigpu.jl`
run(run_cmd);

srun: Job 27855189 step creation temporarily disabled, retrying (Requested nodes are busy)
srun: Step created for StepId=27855189.1


rank=3 rank_loc=3 (gpu_id=CuDevice(3)), size=4, dst=0, src=2
rank=0 rank_loc=0 (gpu_id=CuDevice(0)), size=4, dst=1, src=3
rank=1 rank_loc=1 (gpu_id=CuDevice(1)), size=4, dst=2, src=0
rank=2 rank_loc=2 (gpu_id=CuDevice(2)), size=4, dst=3, src=1
start sending...
recv_mesg on proc 3: [2.0, 2.0, 2.0, 2.0]
recv_mesg on proc 0: [3.0, 3.0, 3.0, 3.0]
done.
recv_mesg on proc 2: [1.0, 1.0, 1.0, 1.0]
recv_mesg on proc 1: [0.0, 0.0, 0.0, 0.0]
